In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('res.csv', sep=',')
df['num_qubits'] = df['num_qubits'].astype(int)
df.replace(to_replace=["None"], value=0, inplace=True)
df.set_index('num_qubits', inplace=True)
df

In [ ]:
df['time_new_scheme_with_opt'] = df['time_new_scheme_with_opt'].astype(float)
df['time_baseline_O0'] = df['time_baseline_O0'].astype(float)
df['time_baseline_O3'] = df['time_baseline_O3'].astype(float)
df['time_new_scheme_without_opt'] = df['time_new_scheme_without_opt'].astype(float)
df['cx_count_with_opt'] = df['cx_count_with_opt'].astype(int)
df['cx_count_without_opt'] = df['cx_count_without_opt'].astype(int)
df['cx_count_baseline_O0'] = df['cx_count_baseline_O0'].astype(int)
df['cx_count_baseline_O3'] = df['cx_count_baseline_O3'].astype(int)

df["time_ratio_with_opt_O0"] = df["time_new_scheme_with_opt"]/df["time_baseline_O0"]
df["time_ratio_with_opt_O3"] = df["time_new_scheme_with_opt"]/df["time_baseline_O3"]
#df["time_ratio_without_opt"] = df["time_new_scheme_without_opt"]/df["time_baseline"]
df["cx_ratio_with_opt_O0"] = df['cx_count_with_opt'].divide(df['cx_count_baseline_O0'])
df["cx_ratio_with_opt_O3"] = df['cx_count_with_opt'].divide(df['cx_count_baseline_O3'])
#df["cx_ratio_without_opt"] = df['cx_count_without_opt'].divide(df['cx_count_baseline'])
df

In [ ]:
#df.groupby(['considered_following_qubits', 'sample_probability'])['cx_ratio_with_opt'].plot(legend=True);

In [ ]:
#df.groupby(['considered_following_qubits', 'sample_probability'])['time_ratio_with_opt'].plot(legend=True);

In [ ]:
x = df#[df.considered_following_qubits==1]
x = x[x.index%5==0][[
    "considered_following_qubits",
    "sample_probability",
    "time_baseline_O0",
    "time_baseline_O3",
    "time_new_scheme_with_opt",
    "time_ratio_with_opt_O0", 
    "time_ratio_with_opt_O3", 
    "cx_count_baseline_O0", 
    "cx_count_baseline_O3", 
    "cx_count_with_opt",
    "cx_ratio_with_opt_O0",
    "cx_ratio_with_opt_O3",
]
]

x["time_baseline_O0"] = x["time_baseline_O0"].round(4)
x["time_baseline_O3"] = x["time_baseline_O3"].round(4)
x["time_new_scheme_with_opt"] = x["time_new_scheme_with_opt"].round(4)
x["cx_ratio_with_opt_O0"] = x["cx_ratio_with_opt_O0"].round(2)
x["cx_ratio_with_opt_O3"] = x["cx_ratio_with_opt_O3"].round(2)
x["time_ratio_with_opt_O0"] = x["time_ratio_with_opt_O0"].round(4)
x["time_ratio_with_opt_O3"] = x["time_ratio_with_opt_O3"].round(4)
x

In [ ]:
x["Benchmark"] = "Qubits=" + x.index.astype(str) + ", p=" + x.sample_probability.astype(str)

In [ ]:
x = x.sort_values(["considered_following_qubits","sample_probability", "num_qubits"])
x.set_index('Benchmark', inplace=True)
x = x.drop('sample_probability', axis=1)
x

In [ ]:
x = x.rename(columns={"time_ratio_with_opt_O0": "Time Ratio O0", 
                      "time_ratio_with_opt_O3": "Time Ratio O3", 
                      "cx_count_baseline_O0": "CX Count Baseline O0",
                      "cx_count_baseline_O3": "CX Count Baseline O3", 
                      "cx_count_with_opt": "CX Count New Scheme", 
                      "cx_ratio_with_opt_O0": "CX Count Ratio O0", 
                      "cx_ratio_with_opt_O3": "CX Count Ratio O3", 
                      "time_new_scheme_with_opt":"Time New Scheme", 
                      "time_baseline_O0":"Time Baseline O0",
                      "time_baseline_O3":"Time Baseline O3"
                     }
            )

In [ ]:
print(x.to_latex())

# Graph Trial

In [ ]:
for considered_following_qubits in [1,1000]:
    for sample_probability in [0.3,0.7]:
        df_new = df[df.considered_following_qubits==1000]
        df_new = df_new[df_new.sample_probability==0.3]

        ax1 = df.plot(kind='scatter', x='time_new_scheme_with_opt', y='cx_count_with_opt', color='r', label="Opt")   
        ax2 = df.plot(kind='scatter', x='time_new_scheme_without_opt', y='cx_count_without_opt', color='black', ax=ax1, label="No Opt")    
        ax3 = df.plot(kind='scatter', x='time_baseline_O0', y='cx_count_baseline_O0', color='gold', ax=ax1, label="Baseline O0")
        ax4 = df.plot(kind='scatter', x='time_baseline_O3', y='cx_count_baseline_O3', color='green', ax=ax1, label="Baseline O3") 
        
        plt.xlabel("Time")
        plt.ylabel("CX Count")
        plt.xscale("log")
        plt.yscale("log")
        plt.legend()
        plt.title("Considered Qubits: " + str(considered_following_qubits) + ", Probability: " + str(sample_probability))
        plt.show()
        
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        ax.scatter(df_new.index, df_new.time_new_scheme_with_opt, df_new.cx_count_with_opt, label="Opt")
        ax.scatter(df_new.index, df_new.time_new_scheme_without_opt, df_new.cx_count_without_opt, label="No Opt")
        ax.scatter(df_new.index, df_new.time_baseline_O0, df_new.cx_count_baseline_O0, label="Baseline O0")
        ax.scatter(df_new.index, df_new.time_baseline_O3, df_new.cx_count_baseline_O3,label="Baseline O3")
        ax.set_xlabel('# Qubits')
        ax.set_ylabel('Time')
        ax.set_zlabel('CX Count')
        ax.legend()
        plt.show()

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.plot(df_new.index, df_new.time_new_scheme_with_opt, df_new.cx_count_with_opt, label="Opt")
ax.plot(df_new.index, df_new.time_new_scheme_without_opt, df_new.cx_count_without_opt, label="No Opt")
ax.plot(df_new.index, df_new.time_baseline_O0, df_new.cx_count_baseline_O0, label="Baseline O0")
ax.plot(df_new.index, df_new.time_baseline_O3, df_new.cx_count_baseline_O3,label="Baseline O3")
ax.set_xlabel('# Qubits')
ax.set_ylabel('Time')
ax.set_zlabel('CX Count')
ax.legend()

plt.show()
